In [1]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/large_merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/all_large_predictions_zero_shot.csv')
print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/all_large_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/large_merged_data.csv
Zero shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/all_large_predictions_zero_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/all_large_1_test_predictions_few_shot.csv


# Data Overview

In [3]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9767 entries, 0 to 9766
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          9767 non-null   float64
 1   verified        9767 non-null   bool   
 2   reviewTime      9767 non-null   object 
 3   reviewerID      9767 non-null   object 
 4   asin            9767 non-null   object 
 5   reviewerName    9767 non-null   object 
 6   reviewText      9759 non-null   object 
 7   summary         9759 non-null   object 
 8   unixReviewTime  9767 non-null   object 
 9   vote            1487 non-null   object 
 10  style           6768 non-null   object 
 11  category        9767 non-null   object 
 12  tech1           2 non-null      object 
 13  description     9767 non-null   object 
 14  fit             0 non-null      float64
 15  title           9767 non-null   object 
 16  also_buy        9767 non-null   object 
 17  tech2           0 non-null      f

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,1.0,False,2015-08-25,A2RYSCZOPEXOCQ,9790787006,The Cat Next Door,"I use a lot of perfume, I go through a new bot...",This is not going to be my favorite scent.,2015-08-25,NaN,...,NaN,Jenna Jameson,[],298.0,"['B00357FTX8', 'B01NBID7FJ', 'B0017JT658']","{'Shipping Weight:': '12.8 ounces (', 'ASIN: '...",All Beauty,NaN,NaN,13.85
1,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
2,5.0,False,2001-06-08,A141OPVE376YFI,B000050B65,Paul G.,"First, a little background. I've switched bet...","Finally, a razor that lives up to the ads",2001-06-08,81,...,NaN,Norelco,[],2.0,"['B01B1O9DOM', 'B00JITDVD2', 'B01KXV16DK', 'B0...",{},All Beauty,NaN,NaN,NaN
3,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50
4,5.0,True,2008-07-25,A1TVTDKNMSQ7XU,B000050B6B,Grandpa Pipes,I've had many Norelco razors in my 50 years of...,Just like new.....,2008-07-25,NaN,...,NaN,Philips Norelco,[],148.0,"['B001IA0PCY', 'B00196W5S4', 'B004URZADG', 'B0...",{'\n Product Dimensions: \n ': '5.1 x 0....,All Beauty,NaN,NaN,64.50


# Zero-shot (OpenAI API)

In [4]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=ITEM_SIDE,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=ZERO_SHOT_SAVE_PATH)

System Fingerprint: fp_eeff13170a
Processing item 1/9767

Details: Jenna Jameson Heartbreaker Perfume for women 3.4 oz Eau De Parfum Spray

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 2/9767

Details: Norelco 6885XL Deluxe Quadra Action Cord/Cordless Rechargeable Men's Shaver

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 3/9767

Details: Norelco 6885XL Deluxe Quadra Action Cord/Cordless Rechargeable Men's Shaver

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 4/9767

Details: Philips Norelco HQ5 Shaving Heads

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 5/9767

Details: Philips Norelco HQ5 Shaving Heads

Predicted Rating: 1.0 stars

----------------

System Fingerprint: fp_eeff13170a
Processing item 6/9767

Details: Braun Clean &amp; Renew Refill Cartridges CCR - 2 Count (Packaging

In [ ]:
# Evaluate Zero-shot Model
evaluate_model_predictions_rmse_mae(
    data_path=ZERO_SHOT_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='rating',
    predicted_ratings_column='zero_shot_predicted_rating'
)

# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

# 1 observation per reviewer - Few-shot OpenAI

In [ ]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=ITEM_SIDE + USER_SIDE + INTERACTION_SIDE,
                                       columns_for_prediction=ITEM_SIDE,
                                       obs_per_user=1,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=FEW_SHOT_1_OBS_SAVE_PATH)


In [ ]:
# Evaluate Few-shot Model
evaluate_model_predictions_rmse_mae(
    data_path=FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='few_shot_predicted_rating'
)


# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a large number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication